# Segmenting and Clustering Neighborhoods in Toronto

In [3]:
# Libraries
from bs4 import BeautifulSoup
import requests

In [4]:
source = requests.get('https://en.m.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [5]:
soup = BeautifulSoup(source,'lxml')

# To get column headers
headers = soup.find('tr').text.split('\n')
headers.remove('')
headers

# To get all the rows
rows = []
for tr in soup.find_all('tr')[1:]:
    try:
        tds = tr.find_all('td')
        x = tds[0].text
        y = tds[1].text
        z = tds[2].text.strip('\n')
        rows.append([x,y,z])
        
    except IndexError:
        continue

In [6]:
import pandas as pd
df = pd.DataFrame(rows, columns =headers)

In [7]:
# To drop the Not assigned Boroughs
df = df[df.Borough != 'Not assigned']

# Reset the index
df.reset_index(inplace = True)
del df['index']

# Groupby Postal Codes
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [8]:
 for i,j in zip(df['Borough'],df['Neighbourhood']):
        if j == 'Not assigned':
            j = i

In [9]:
df_gr = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join)

In [10]:
df_gr = pd.DataFrame(df_gr).reset_index()

In [11]:
df_gr.head(12)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [15]:
df_gr.shape

(103, 3)

In [25]:
df_coordinates = pd.read_csv('Geospatial_Coordinates.csv')
df_coordinates = df_coordinates.rename(columns={"Postal Code": "Postcode"})
df_coordinates.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [27]:
df_final = pd.merge(df_gr, df_coordinates, on=['Postcode'], how='inner')
df_final.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
